## Portfolio Return Summary

In [1]:
# Import initial libraries
from ScrapingData.Operations_3 import *
from Scraping_Fund_Parameter import *
import warnings

import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import plotly.graph_objs as go

# Leveraged from the helpful Datacamp Python Finance trading blog post.
#from pandas_datareader import data as pdr
#import yfinance as yf

%matplotlib inline

# Imports in order to be able to use Plotly offline.
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#print(__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)
# Import the Sample worksheet with acquisition dates and initial cost basis:


warnings.filterwarnings("ignore")

## Uncomment to control decimal digits in the dataframe
pd.options.display.float_format = '{:.1f}'.format

##### This variable is used for YTD performance.
last_year = datetime.datetime.now() - relativedelta(years=1)
# Create string of month name and year...
text = format(last_year, '%Y')
end_of_last_year = datetime.datetime(int(text), 12, 31)


### Read data from Google Sheet
readSheet=ReadSheet()
sheetFList=readSheet.Authorization_Fund()



def CreateMergedDataframe(colString, inDict, mergedDf, fList):
    dummyList=[]
    for n in fList:
        dummyList.append(inDict[n])

    dummyDf=pd.DataFrame(dummyList, columns=[colString])
    mergedDf=pd.merge(mergedDf, dummyDf, left_index=True, right_index=True)
    del dummyList, dummyDf
    return mergedDf

def GetTickerDate(sheetFList,end_of_last_year):
    fundDict=dict(zip(fundList, fundCost))
    fundOriginDateDict=dict(zip(fundList,fundOriginDate))
    fundOriginVolumeDict=dict(zip(fundList, fundOrigin))
    #print(' funddate : ',fundOriginDateDict)

    dfList=[]
    for n in sheetFList:
        dfIn=readSheet.LoadSheet(n)
        dfList.append(dfIn)
    
    mainList=[]
    minorList=[]
    for n in dfList:
        dummy=(n[(n['Date']<=end_of_last_year)].tail(1))
        #print(' :: ',n, ' -- ',dummy)
        if(len(dummy)>0):
            mainList.append(dummy['Price'].values[0])
            minorList.append(dummy['Date'].values[0])
        else:
            mainList.append(0.0)
            minorList.append(end_of_last_year)
    fundEndLastYearDict=dict(zip(fundList,mainList))
    dateEndLastYearDict=dict(zip(fundList,minorList))
    # for n in dfList:
    #     print(n)

    resultList=[]
    latestList=[]
    for n in dfList:
        #print(n, ' :: ',len(n))
        resultList.append(float(n.loc[len(n)-1, 'Price']))
        updateDate=datetime.datetime.strptime(n.loc[len(n)-1, 'Update'],'%d/%m/%y')    
        latestList.append(updateDate.strftime("%Y-%m-%d"))
    resultDict=dict(zip(fundList,resultList))
    latestUpdateDict=dict(zip(fundList,latestList))

    
    ### Get % fund
    originDict=dict(zip(fundList, fundOrigin))
    totalUnitList=[]
    updatePriceList=[]
    sum=0
    for n in fundList:
        totalUnitList.append(originDict[n]/fundDict[n])
        updatePriceList.append(resultDict[n]*originDict[n]/fundDict[n])
        sum=sum+(resultDict[n]*originDict[n]/fundDict[n])

    percentFundList=[]
    for n in fundList:
        percentFundList.append((100*resultDict[n]*originDict[n]/fundDict[n])/sum)

    #totalUnitListDict=dict(zip(fundList,totalUnitList))
    updatePriceDict=dict(zip(fundList,updatePriceList))
    percentFundDict=dict(zip(fundList,percentFundList))    
    
    ####
    percentDiffList=[]
    for n in fundList:
        #print(n, ' :: ',type(fundDict[n]), ' :: ',type(resultDict[n]))
        dummy=(resultDict[n]-fundDict[n])/fundDict[n]
        percentDiffList.append(dummy)

    diffDict=dict(zip(fundList,percentDiffList))
    
    
    displayOutput=0
    if(displayOutput==1):
        print(fundDict)
        print(fundOriginDateDict)
        print(fundOriginVolumeDict)
        print(resultDict)
        print(latestUpdateDict)
        print(fundEndLastYearDict)
        print(dateEndLastYearDict)
        print(percentFundDict)

    del dfList, mainList, resultList, updateDate, latestList, minorList, totalUnitList, updatePriceList, percentFundList, originDict, percentDiffList
    return fundOriginDateDict, fundOriginVolumeDict, fundDict, dateEndLastYearDict, fundEndLastYearDict, latestUpdateDict, resultDict, percentFundDict, diffDict, updatePriceDict

fundOriginDateDict, fundOriginVolumeDict, fundDict, dateEndLastYearDict, fundEndLastYearDict, latestUpdateDict, resultDict, percentFundDict, diffDict, updatePriceDict=GetTickerDate(sheetFList,end_of_last_year)

totalReturn=0
totalUpdateFund=0
for n in fundList:
    #print(diffDict[n],' :: ',percentFundDict[n])
    totalReturn=totalReturn+100*(0.01*diffDict[n]*0.01*percentFundDict[n])
    totalUpdateFund=totalUpdateFund+updatePriceDict[n]



C:\Users\70018928\AppData\Local\Continuum\anaconda3\envs\quandl\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
merged_portfolio=pd.DataFrame(fundList, columns=['Ticker'])
merged_portfolio=CreateMergedDataframe('AcquisitionDate', fundOriginDateDict, merged_portfolio, fundList)
merged_portfolio=CreateMergedDataframe('CostBasis', fundOriginVolumeDict, merged_portfolio, fundList)
merged_portfolio=CreateMergedDataframe('UnitCost', fundDict, merged_portfolio, fundList)
merged_portfolio['Quantity']=merged_portfolio['CostBasis']/merged_portfolio['UnitCost']
merged_portfolio=CreateMergedDataframe('StartOfYear', dateEndLastYearDict, merged_portfolio, fundList)
merged_portfolio=CreateMergedDataframe('StartOfYearPrice', fundEndLastYearDict, merged_portfolio, fundList)
merged_portfolio=CreateMergedDataframe('Date', latestUpdateDict, merged_portfolio, fundList)
merged_portfolio=CreateMergedDataframe('Close', resultDict, merged_portfolio, fundList)
merged_portfolio=CreateMergedDataframe('PercentFund', percentFundDict, merged_portfolio, fundList)
merged_portfolio=merged_portfolio.set_index(['Ticker'])
merged_portfolio['TotalReturn']=merged_portfolio['Close']/merged_portfolio['UnitCost']-1
merged_portfolio['YTDReturn']=merged_portfolio['Close']/merged_portfolio['StartOfYearPrice']-1
merged_portfolio['UpdateFund']=merged_portfolio['Close']*merged_portfolio['Quantity']
merged_portfolio['YTDValue']=(merged_portfolio['Close']-merged_portfolio['StartOfYearPrice'])*merged_portfolio['Quantity']
totalYTD=merged_portfolio['YTDValue'].sum(axis=0, skipna=True)

del fundOriginVolumeDict, fundDict, dateEndLastYearDict, fundEndLastYearDict, resultDict, percentFundDict

#display(merged_portfolio)
show_portfolio=merged_portfolio[['Date','TotalReturn','YTDReturn','PercentFund','CostBasis','UpdateFund']]
show_portfolio['TotalReturn']=100*show_portfolio['TotalReturn']
show_portfolio['YTDReturn']=100*show_portfolio['YTDReturn']
show_portfolio.columns=['Date','%Total Return','%YTD Return','%Fund','Cost','Latest']
display(show_portfolio)

,Date,%Total Return,%YTD Return,%Fund,Cost,Latest
Ticker,,,,,,
TMBCOF,2021-02-24,11.4,8.8,23.0,60000.0,66863.6
ONE-UGG-RA,2021-02-24,16.0,7.5,23.9,60000.0,69580.1
K-USA-A(A),2021-02-24,19.6,3.3,25.3,61490.3,73524.8
KFGBRAND-A,2021-02-24,-0.5,-1.9,13.7,40000.0,39792.1
WE-CHIG,2021-02-24,14.2,6.8,7.8,20000.0,22840.1
WE-CYBER,2021-02-24,-7.9,-7.9,6.3,20000.0,18426.3


In [3]:
print("************************************************")
print(f"Percent of Total Return : {100*totalReturn: .2f}        %")
print(f"Update Fund             : {totalUpdateFund: ,.1f} THB")
print("************************************************")
print(f"YTD Gain                : {totalYTD: ,.1f} THB")
print("************************************************")
del show_portfolio, totalReturn, totalUpdateFund, totalYTD

************************************************
Percent of Total Return :  11.94        %
Update Fund             :  291,027.0 THB
************************************************
YTD Gain                :  11,646.3 THB
************************************************


In [4]:
## Load benchmarkingData from Google Sheet
sheetBList=readSheet.Authorization_Benchmark()
sheetSList=readSheet.Authorization_Benchmark_Scaped()

dfList=[]
for n in sheetBList:
    dfIn=readSheet.LoadSheet(n)
    dfList.append(dfIn)
for n in sheetSList:
    dfIn=readSheet.LoadSheet(n)
    dfList.append(dfIn)
   
    

#print(fundOriginDateDict)
#print(benchmarkPairDict)
#print(latestUpdateDict)

benchmarkOriginDateList=[]
benchmarkUpdateList=[]
for n in benchmarkPairDict:    
    #print(' ==> ',n)
    benchmarkOriginDateList.append(fundOriginDateDict[n])
    benchmarkUpdateList.append(latestUpdateDict[n])
    
#benchmarkOriginDateDict=dict(zip(benchmarkList,benchmarkOriginDateList))
benchmarkOriginDateDict=dict(zip(list(benchmarkPairDict),benchmarkOriginDateList))
benchmarkUpdateDict=dict(zip(list(benchmarkPairDict), benchmarkUpdateList))

del benchmarkOriginDateList, benchmarkUpdateList

#print(benchmarkOriginDateDict)
#print(benchmarkUpdateDict)

In [5]:
benchmarkTotalList=BenchmarkList+BenchmarkScrapeList
benchmarkDataDict=dict(zip(benchmarkTotalList, dfList))

date_mainList=[]
date_minorList=[]
date_ytdList=[]
price_mainList=[]
price_minorList=[]
price_ytdList=[]
#print(benchmarkDataDict)
#print(benchmarkOriginDateDict)
for n in benchmarkOriginDateDict:    
    #print(n,' :: ',benchmarkOriginDateDict[n], ' :: ',n)
    dummyDf=benchmarkDataDict[benchmarkPairDict[n]]
    #print(dummyDf, ' == ',dummyDf.columns)
    dummy=dummyDf[(dummyDf['Date']>=benchmarkOriginDateDict[n]) & (dummyDf['Date']<=benchmarkUpdateDict[n])]
    #print(' ==> ',dummy)
    date_mainList.append(dummy['Date'].head(1).values[0])
    date_minorList.append(dummy['Date'].tail(1).values[0])
    price_mainList.append(dummy['Price'].head(1).values[0])
    price_minorList.append(dummy['Price'].tail(1).values[0])
    dummy=(dummyDf[(dummyDf['Date']<=end_of_last_year)].tail(1))
    date_ytdList.append(dummy['Date'].values[0])
    price_ytdList.append(dummy['Price'].values[0])
    
#print(date_mainList)  # acquisition date
# print(price_mainList)  
# print(date_minorList)  # latest date
# print(price_minorList)
# print(date_ytdList)   # start of year date
# print(price_ytdList)

#print(benchmarkPairDict)

acquisitionDateDict=dict(zip(benchmarkList, date_mainList)) 
acquisitionPriceDict=dict(zip(benchmarkList, price_mainList)) 
latestDateDict=dict(zip(benchmarkList, date_minorList)) 
latestPriceDict=dict(zip(benchmarkList, price_minorList)) 
ytdDateDict=dict(zip(benchmarkList, date_ytdList)) 
ytdPriceDict=dict(zip(benchmarkList, price_ytdList)) 

#print(' ac date: ', acquisitionDateDict)

del dfList
del date_mainList, price_mainList, date_minorList, price_minorList, date_ytdList, price_ytdList
del benchmarkDataDict, benchmarkOriginDateDict, benchmarkUpdateDict

In [6]:
benchmark_portfolio=pd.DataFrame(list(benchmarkPairDict), columns=['Ticker'])
benchmark_portfolio=CreateMergedDataframe('BM_AcquisitionDate',acquisitionDateDict,benchmark_portfolio,list(acquisitionDateDict))
benchmark_portfolio=CreateMergedDataframe('BM_AcquisitionPrice',acquisitionPriceDict,benchmark_portfolio,list(acquisitionDateDict))    
benchmark_portfolio=CreateMergedDataframe('BM_LatestDate',latestDateDict,benchmark_portfolio,list(acquisitionDateDict))
benchmark_portfolio=CreateMergedDataframe('BM_LatestPrice',latestPriceDict,benchmark_portfolio,list(acquisitionDateDict))    
benchmark_portfolio=CreateMergedDataframe('BM_StartOfYear',ytdDateDict,benchmark_portfolio,list(acquisitionDateDict))
benchmark_portfolio=CreateMergedDataframe('BM_StartOfYear_Price',ytdPriceDict,benchmark_portfolio,list(acquisitionDateDict))    

del acquisitionDateDict, acquisitionPriceDict, latestDateDict, latestPriceDict, ytdDateDict, ytdPriceDict

#benchmark_portfolio

In [7]:
merged_portfolio.reset_index()
#final_portfolio=pd.merge(merged_portfolio, benchmark_portfolio, on='Ticker')
final_portfolio=pd.concat([merged_portfolio, benchmark_portfolio.set_index("Ticker")], axis=1, join="outer")
del benchmark_portfolio

#final_portfolio

In [8]:
final_portfolio['EqBMShares'] = final_portfolio['CostBasis'] / (final_portfolio['BM_AcquisitionPrice']/30.0)

# Percent return of SP from acquisition date of position through latest trading day.
final_portfolio['BM_Return'] = final_portfolio['BM_LatestPrice'] / final_portfolio['BM_AcquisitionPrice'] - 1

# This is a new column which takes the tickers return and subtracts the sp 500 equivalent range return.
final_portfolio['AbsReturnCompare'] = final_portfolio['TotalReturn'] - final_portfolio['BM_Return']

# This is a new column where we calculate the ticker's share value by multiplying the original quantity by the latest close.
final_portfolio['TickerShareValue'] = final_portfolio['Quantity'] * final_portfolio['Close']

# We calculate the equivalent SP 500 Value if we take the original SP shares * the latest SP 500 share price.
final_portfolio['BM_Value'] = final_portfolio['EqBMShares'] * final_portfolio['BM_LatestPrice']/30.0

# This is a new column where we take the current market value for the shares and subtract the SP 500 value.
final_portfolio['AbsValueCompare'] = final_portfolio['TickerShareValue'] - final_portfolio['BM_Value']/30.0

# This column calculates profit / loss for stock position.
final_portfolio['Fund_Gain(Loss)'] = final_portfolio['TickerShareValue'] - final_portfolio['CostBasis']

# This column calculates profit / loss for SP 500.
#final_portfolio['BM_Gain(Loss)'] = final_portfolio['BM_Value'] - final_portfolio['CostBasis']
final_portfolio['BM_Gain(Loss)'] = (final_portfolio['BM_Value'] - ((1/30.0)*final_portfolio['EqBMShares']*final_portfolio['BM_AcquisitionPrice']))

# YTD return for SP to run compares.
final_portfolio['BM_YTD'] = final_portfolio['BM_LatestPrice'] / final_portfolio['BM_StartOfYear_Price'] - 1

# Cumulative sum of original investment
final_portfolio['Cum_Invst'] = final_portfolio['CostBasis'].cumsum()

# Cumulative sum of Ticker Share Value (latest FMV based on initial quantity purchased).
final_portfolio['Cum_Ticker_Returns'] = final_portfolio['TickerShareValue'].cumsum()

# Cumulative sum of SP Share Value (latest FMV driven off of initial SP equiv purchase).
final_portfolio['Cum_BM_Returns'] = final_portfolio['BM_Value'].cumsum()

# Cumulative CoC multiple return for stock investments
final_portfolio['Cum_Ticker_ROI_Mult'] = final_portfolio['Cum_Ticker_Returns'] / final_portfolio['Cum_Invst']


final_portfolio=final_portfolio.reset_index()
final_portfolio=final_portfolio.rename(columns={"index":"Ticker"})
#final_portfolio


In [9]:
trace1 = go.Bar(
    x = final_portfolio['Ticker'][0:10],
    y = final_portfolio['TotalReturn'][0:10],
    name = 'Ticker Total Return')
trace2 = go.Bar(
    x = final_portfolio['Ticker'][0:10],
    y = final_portfolio['YTDReturn'][0:10],
    name = 'Ticker YTD Return')
trace3 = go.Scatter(
    x = final_portfolio['Ticker'][0:10],
    y = final_portfolio['BM_Return'][0:10],
    marker_size=20,
    name = 'Benchmarker Total Return')
trace4 = go.Scatter(
    x = final_portfolio['Ticker'][0:10],
    y = final_portfolio['BM_YTD'][0:10],
    marker_size=20,
    name = 'Benchmarker YTD Return')
    
data = [trace1, trace2, trace3 , trace4]
layout = go.Layout(title = 'Total vs Benchmarker Return '
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [10]:
trace1 = go.Bar(
    x = final_portfolio['Ticker'][0:10],
    y = final_portfolio['Cum_Invst'][0:10],
    name = 'Cum_Invst')
trace2 = go.Bar(
    x = final_portfolio['Ticker'][0:10],
    y = final_portfolio['Cum_BM_Returns'][0:10],
    name = 'Cum_BM_Returns')
trace3 = go.Bar(
    x = final_portfolio['Ticker'][0:10],
    y = final_portfolio['Cum_Ticker_Returns'][0:10],
    name = 'Cum_Ticker_Returns')

trace4 = go.Scatter(
    x = final_portfolio['Ticker'][0:10],
    y = final_portfolio['Cum_Ticker_ROI_Mult'][0:10],
    yaxis='y2',
    marker_size=20,
    name = 'Cum ROI Mult')
    
data = [trace1, trace2, trace3, trace4]
layout = go.Layout(title = 'Total Cumulative Investment Over Time'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".1")
    , yaxis2= {'title':'ROI', 'overlaying':'y','side':'right' }
    , xaxis=dict(title='Ticker')
    , legend=dict(xanchor='left',x=.01,yanchor='top',y=1)
    )
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [11]:
trace1 = go.Bar(
    x = final_portfolio['Ticker'][0:10],
    y = final_portfolio['Fund_Gain(Loss)'][0:10],
    name = 'Total Return (THB)')
trace2 = go.Bar(
    x = final_portfolio['Ticker'][0:10],
    y = final_portfolio['BM_Gain(Loss)'][0:10],
    name = 'Benchmarker Return (THB)')

trace3 = go.Scatter(
    x = final_portfolio['Ticker'][0:10],
    y = final_portfolio['TotalReturn'][0:10],
    yaxis='y2',
    marker_size=20,
    name = 'Total Return %')
trace4 = go.Scatter(
    x = final_portfolio['Ticker'][0:10],
    y = final_portfolio['BM_Return'][0:10],
    yaxis='y2',
    marker_size=20,
    name = 'BM Return %')


data = [trace1, trace2, trace3, trace4]
layout = go.Layout(title = 'Gain/ (Loss) Total vs Benchmarker Return'
    , barmode = 'group'
    , yaxis=dict(title='Gain/ (Loss) THB', tickformat=".1")
    , yaxis2= {'title':'Ticker Return', 'overlaying':'y','side':'right' , 'tickformat':".2%" }
    , xaxis=dict(title='Ticker')
    , legend=dict(xanchor='left',x=.7,yanchor='top',y=1)
    )
fig = go.Figure(data=data, layout=layout)
iplot(fig)